In [10]:
import pandas as pd
import openai
import os
import re
from dotenv import load_dotenv

In [11]:
api_key = os.environ['GPT_OPEN_AI_KEY']

In [12]:
xlsx_obj = pd.read_excel("tour.xlsx")

In [13]:
xlsx_obj.head()

,tour_id,operator_id,enquiry_id,user_id,channel,author,type,added_at,message
0,14263,1170,825562,1645868,customer,traveller,message,2023-01-01 22:28:48,"- Automatic Message -\n\nIf I purchase, Europe..."
1,14263,1170,825975,1646704,operator,operator,message,2023-01-09 08:22:52,<i>Traveller's question for TourRadar</i>\n\nC...
2,14263,1170,825975,1646703,customer,traveller,message,2023-01-09 19:58:06,"Good afternoon , I am already booked on the to..."
3,14263,1170,826114,1647006,customer,traveller,message,2023-01-02 21:33:30,- Automatic Message -\n\nHi there!\n\nCould yo...
4,14263,1170,826114,1647007,customer,operator,message,2023-01-05 06:18:40,"Hi Karen\n\nThank you for reaching out to us,\..."


In [14]:
xlsx_obj['tour_id'].unique()

array([14263], dtype=int64)

In [15]:
def replace_html_with_newline(text):
    # Replace common HTML tags with newline
    text = re.sub(r'<br\s*/?>', '\n', text)
    text = re.sub(r'</p>', '\n', text)
    text = re.sub(r'<[^>]+>', '', text)  # Remove any other HTML tags
    return text

In [16]:
def enclose_last_column_in_quotes(df):
    last_column = df.columns[-1]
    df[last_column] = df[last_column].apply(lambda x: f'"{x}"' if isinstance(x, str) else x)
    return df

In [1]:
def summarize_messages(messages):
    # Example summarization function that trims messages to a certain length
    return [msg[:200] + '...' if len(msg) > 200 else msg for msg in messages]

In [22]:
def process_excel_to_text(excel_file_path, columns):
    # Read the specified columns from the Excel file
    df = pd.read_excel(excel_file_path, usecols=columns)
    
    # Replace HTML content with newlines in the 'message' column
    df['message'] = df['message'].apply(lambda x: replace_html_with_newline(x) if isinstance(x, str) else x)
    
    # Enclose the 'message' column in double quotes
    df = enclose_last_column_in_quotes(df)
    
    # Convert DataFrame to CSV format in text
    csv_text = df.to_csv(index=False)
    return csv_text

In [24]:
excel_file_path = 'tour.xlsx'  # Path to your Excel file
columns = ['tour_id', 'message']  # Columns to extract
csv_text = process_excel_to_text(excel_file_path, columns)

print("CSV formatted text:")
print(csv_text)

# Use the CSV text as input to GPT-3
prompt_csv = f"Summarize the following CSV data:\n\n{csv_text}"
print("\nPrompt for GPT-3:")
print(prompt_csv)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [28]:
prompt=f"""
Below I provided you with a File in a text/csv format.

**File Description**
The file contains columns tour_id and message. The message column includes questions and answers from users and tour organizations.

**Instructions**
1. Identify pairs of related questions and answers within the same tour_id.
2. Exclude unrelated pairs even if the questions and answers seem related but have different tour_id`s.

**Task Description**
1. Generate the top 10 frequently asked question-answer pairs for each unique tour_id.
2. Format the output as FAQs.

**Example Data**
tour_id: 1
message: "Which criteria are used to calculate the pricing"
tour_id: 1
message: "There are a total of five places to visit."
tour_id: 3
message: "There is no cancelation plan for this tour."
tour_id: 1
message: "There are factors like food, transportation, and residential costs that are taken into consideration for the pricing."
tour_id: 1
message: "Is there any cancelation plan?"

**Correct Pairing Example**
Question: "Which criteria are used to calculate the pricing"
Answer: "There are factors like food, transportation, and residential costs that are taken into consideration for the pricing."

**Incorrect Pairing Example**
Question: "Is there any cancelation plan?"
Answer: "There is no cancelation plan for this tour."
because the question belongs to the tour_id: 1 and answer from tour_id: 3

**Expected Output**
For each tour_id, output the top 10 FAQs in the following format:
Question: <question text>
Answer: <answer to the question>

**CSV File data**
{prompt_csv}
"""

In [31]:
messages = [
    {"role": "system", "content": "Hoe can I help you?"},
    {"role": "user", "content": prompt},
]

In [32]:
messages

[{'role': 'system', 'content': 'Hoe can I help you?'},
 {'role': 'user',
  'content': '\nBelow I provided you with a File in a text/csv format.\n\n**File Description**\nThe file contains columns tour_id and message. The message column includes questions and answers from users and tour organizations.\n\n**Instructions**\n1. Identify pairs of related questions and answers within the same tour_id.\n2. Exclude unrelated pairs even if the questions and answers seem related but have different tour_id`s.\n\n**Task Description**\n1. Generate the top 10 frequently asked question-answer pairs for each unique tour_id.\n2. Format the output as FAQs.\n\n**Example Data**\ntour_id: 1\nmessage: "Which criteria are used to calculate the pricing"\ntour_id: 1\nmessage: "There are a total of five places to visit."\ntour_id: 3\nmessage: "There is no cancelation plan for this tour."\ntour_id: 1\nmessage: "There are factors like food, transportation, and residential costs that are taken into consideration fo

In [34]:
from openai import OpenAI
client = OpenAI(api_key=api_key)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=messages
)

print(completion.choices[0].message)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'messages[1].content': string too long. Expected a string with maximum length 1048576, but got a string with length 1657149 instead.", 'type': 'invalid_request_error', 'param': 'messages[1].content', 'code': 'string_above_max_length'}}